In [1]:
import pandas as pd
import numpy as np
df = pd.read_csv(r"./Aleppo2017_processed.csv",encoding='utf-8')
df.head()

    id                 time     gl
0  183  2015-05-16 05:35:41  162.0
1  183  2015-05-16 05:30:41  164.0
2  183  2015-05-16 05:25:41  168.0
3  183  2015-05-16 05:20:41  169.0
4  183  2015-05-16 05:15:41  170.0


In [2]:
ID = df['id'].value_counts().index
print(ID)

Index([263,  77, 193, 277, 229, 245, 155, 251, 111, 164,
       ...
       128, 264, 186, 162, 249,  39,  52, 266, 223, 289],
      dtype='int64', name='id', length=226)


In [3]:
data = df.groupby('id')
pd.set_option('display.max_rows',100)
data.size()

id
2      68055
3      72521
5      54283
7      78663
8      51815
       ...  
289    16439
290    64999
291    62832
292    61514
293    61037
Length: 226, dtype: int64

In [7]:
import datetime

for i in ID[:10]:
    #获取数据
    print("开始获取id为{}的被试的数据：---------------------------------".format(i))
    data = df[df['id']== i].copy()
    
    #将time列设成date数据类型
    data['time'] = pd.to_datetime(data['time'])

    #按照时间排序
    data = data.sort_values(by='time') 
    
    #提取年月日为单独的一列
    data['date'] = data['time'].dt.date
    
    # 使用groupby来统计每天的血糖数据条数
    daily_data_count = data.groupby('date')['gl'].count()
    print(daily_data_count)

    num_day = 0
    for j, count in daily_data_count.items():
        if num_day >= 7:
            break
        else:
            if count < 200:
                print("{}这天的数据量太少，直接删除".format(j), count)
                continue
            else:
                num_day += 1
                day_data = data[data['time'].dt.date == j][['time','id','gl']].copy()


                if day_data.shape[0] == 288:
                    print("血糖记录满足288的日期: ",j)

                    day_data = day_data.set_index('time')

                    day_data.to_csv(r"C:\Users\admin\Desktop\week_of_Data.csv", mode='a', header=False,index=True)

                elif day_data.shape[0] > 200 :
                    print("{}这天的血糖记录数不满足288".format(j))
                    print("day_data.shape",day_data.shape)
                    #设置秒为0
                    day_data.loc[:,'time'] = day_data['time'].apply(lambda dt: dt.replace(second=0))

                    data_day = []
                    for k in range(24):

                        hour_data = day_data[day_data['time'].dt.hour == k].copy()    #某些小时可能没有数据

                        if hour_data.empty:
                            print("数据在 {}-{}  小时时间范围内为空，先全部插值为0".format(k,k+1))

                            start_date = datetime.datetime.combine(j, datetime.time(k, 0))

        #                     print(start_date)

                            full_time_index = pd.date_range(start=start_date, periods=12, freq='5T')

                            data_empty = {'id':[i] * 12,'gl': [0] * 12}

                            data_day_padding = pd.DataFrame(data_empty, index=full_time_index)

                            data_day.append(data_day_padding)

                        else:
                            minute = hour_data.iloc[0]['time'].minute % 5

                            # 设置一小时内的起始时间
                            start_date = hour_data.iloc[0]['time'].replace(hour=k, minute=minute)
            #                 print(start_date)

                            # 使用完整的时间序列重新索引
                            full_time_index = pd.date_range(start=start_date, periods=12, freq='5T')

                            #删除重复时间的记录
                            data_unique_times = hour_data[~hour_data['time'].duplicated(keep='first')]

                            # 将时间戳设置为索引
                            data_unique_times = data_unique_times.set_index('time')

                            # 使用完整的时间序列重新索引
                            hour_data_interpolated = data_unique_times.reindex(index=full_time_index)

                            # 进行线性插值interpolate()    
                            #线性插值的问题：
                            hour_data_interpolated['gl'] = hour_data_interpolated['gl'].interpolate()
                            hour_data_interpolated['id'] = i
                            hour_data_interpolated = hour_data_interpolated.fillna({'gl': 0})
        #                     print("线性插值后的数据:", hour_data_interpolated)
                            data_day.append(hour_data_interpolated)
                    data_day_all = pd.concat(data_day)
                    data_day_all.to_csv(r"C:\Users\admin\Desktop\week_of_Data.csv", mode='a', header=False,index=True)

开始获取id为263的被试的数据：---------------------------------
date
2015-01-04    250
2015-01-05    288
2015-01-06     97
2015-01-08    163
2015-01-09    285
             ... 
2016-02-12    288
2016-02-13    289
2016-02-14    289
2016-02-15    289
2016-02-16    134
Name: gl, Length: 351, dtype: int64
2015-01-04这天的血糖记录数不满足288
day_data.shape (250, 3)
数据在 0-1  小时时间范围内为空，先全部插值为0
数据在 1-2  小时时间范围内为空，先全部插值为0
数据在 2-3  小时时间范围内为空，先全部插值为0
血糖记录满足288的日期:  2015-01-05
2015-01-06这天的数据量太少，直接删除 97
2015-01-08这天的数据量太少，直接删除 163
2015-01-09这天的血糖记录数不满足288
day_data.shape (285, 3)
2015-01-10这天的血糖记录数不满足288
day_data.shape (282, 3)
2015-01-11这天的血糖记录数不满足288
day_data.shape (285, 3)
2015-01-12这天的血糖记录数不满足288
day_data.shape (286, 3)
血糖记录满足288的日期:  2015-01-13
开始获取id为77的被试的数据：---------------------------------
date
2015-02-14     92
2015-02-15    250
2015-02-16    288
2015-02-17    288
2015-02-18    288
             ... 
2016-01-31    292
2016-02-01    233
2016-02-02    299
2016-02-03    566
2016-02-04    187
Name: gl, Length: 342, d